## Создание продвинутых ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Completion API, RAG и Function Calling.

Для начала, установим OpenAI SDK и другие полезные библиотеки:

In [ ]:
%pip install openai pydantic

**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

И ещё пара полезных функций на будущее:

In [3]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

## Авторизация

Для работы с языковыми моделями нам понадобится авторизоваться в Yandex Cloud. Это можно сделать несколькими способами:

* через iam-токен. Для получения iam-токена необходимо создать авторизованный ключ доступа к сервисному аккаунту, и знать `service_accound_id`, `key_id` и `private_key`. Скачайте файл с ключами доступа `authorized_key.json`, и добавьте к нему поле `folder_id`
* **[рекомендованный способ]** через ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere или в переменных окружения. Если вы используете набор переменных окружения в файле `.env` - запустите следующую ячейку для их загрузки.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
import json
import os
from util.iam_auth import get_iam

if os.path.exists('authorized_key.json'):
    with open('authorized_key.json') as f:
        auth_key = json.load(f)

    api_key = get_iam(auth_key['service_account_id'],auth_key['id'],auth_key['private_key'])
    folder_id = auth_key['folder_id']
    print(f"Using IAM Token Auth with folder_id={folder_id}, iam token={api_key}")
else:
    folder_id = None

Using IAM Token Auth with folder_id=b1gst3c7cskk2big5fqn, iam token=t1.9euelZqNkpCPl5WWkM3JzsyOzJfKj-3rnpWakZ2dyI2PzIvGmMaOj5rGjJrl8_cnEzw5-e9fS1Iu_t3z92dBOTn5719LUi7-zef1656VmozIy8jKkprOlYuajcqZjpjN7_zN5_XrnpWaj46ekIzOmM2Rl52JmJzJlorv_cXrnpWajMjLyMqSms6Vi5qNypmOmM0.HMyUZ6f6M7iiGmP0A8GxLiH5D2QTK9HnAkX9UcRZQkzB_eP94ZN1hoF12Q3QNVPP0butCMGRo7cQD-ujic6NCg


Если файл с ключем не найден, то используем значения `api_key` из переменных окружения:

In [24]:
if folder_id is None:
    folder_id = os.environ["folder_id"]
    api_key = os.environ["api_key"]


Создадим какую-нибудь модель из Foundation Models и убедимся, что она кое-что знает про вина. 

> ВНИМАНИЕ: Для правильной работы необходимо передать `folder_id` в параметр `project` при создании объекта OpenAI SDK.

In [49]:
import os
from openai import OpenAI

model = f"gpt://{folder_id}/yandexgpt/latest"
model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = OpenAI(
    base_url="https://rest-assistant.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

In [50]:
res = client.responses.create(
    model = model,
    input = "Какое вино можно пить со стейком?"
)

printx(res.output_text)

Сочетание вина со стейком — это классика, и выбор зависит от вида мяса, способа приготовления и личных предпочтений. Вот основные рекомендации:

### 1. **Красные вина — лучший выбор**
Стейк (особенно из говядины) имеет насыщенный, мясной вкус и жирность, поэтому он хорошо сочетается с полновкусными красными винами.

---

### 🍷 **Лучшие вина к стейку:**

#### 🔹 **Каберне Совиньон (Cabernet Sauvignon)**
- **Почему подходит:** Выразительная структура, танины и кислотность "обволакивают" жир стейка, очищая вкус.
- **Ноты:** чёрная смородина, кедр, перец, табак.
- **Идеально:** к рибай, томахоку, веллингтону.

#### 🔹 **Сира / Шираз (Syrah / Shiraz)**
- **Почему подходит:** насыщенный, пряный, с дымком — отлично с жареным мясом.
- **Ноты:** чёрные ягоды, дёготь, перец, дым.
- **Идеально:** к стейкам с дымным ароматом (гриль, барбекю).

#### 🔹 **Мальбек (Malbec)**
- **Почему подходит:** мягкие танины, сочная фруктовость, бархатистость.
- **Ноты:** чёрная вишня, шоколад, ваниль.
- **Идеально:** к аргентинскому стейку, бифштексу.

#### 🔹 **Неббиоло (Nebbiolo) — например, Бароло**
- **Почему подходит:** мощное, структурированное, с высокой кислотностью.
- **Ноты:** вишня, роза, грибы, кожа.
- **Идеально:** к выдержанному стейку или веллингтону.

#### 🔹 **Зинфандель (Zinfandel) — особенно калифорнийский**
- **Почему подходит:** сладковатые фруктовые ноты и высокий градус хорошо балансируют жирность.
- **Ноты:** малина, инжир, специи.
- **Идеально:** к стейку с соусом барбекю.

---

### ❌ Что лучше избегать:
- **Лёгкие вина** (Пино Нуар, Божоле) — могут "потеряться" на фоне стейка.
- **Белые вина** — редко, но если очень хочется, выбирайте насыщенные, выдержанные в дубе: например, **Шардоне** (Chardonnay) с дымком и маслянистостью — только к лёгким стейкам (филе-миньон) и с соусом.

---

### 🔥 Бонус: а если стейк с соусом?
- **Соус песто** → Легкий Сира или Мерло.
- **Соус пеппер (перечный)** → Каберне Совиньон или Шираз.
- **Соус барбекю** → Зинфандель или сладковатый Мальбек.

---

### 💡 Совет:
Если сомневаетесь — **Каберне Совиньон** — самый универсальный и безопасный выбор к стейку.

Хорошего ужина! 🥩🍷

## Responses API

Когда мы используем запрос вида `client.responses.create` - мы используем так называемый Responses API. Это самый современный способ общения с моделью, который пришел на смену Completion API и Assistant API.

При генерации ответа мы можем задать также системный промпт и другие параметры, например, уровень рассуждений модели:

In [27]:
res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    instructions = "Ты - опытный сомелье, задача которого - консультировать пользователя в вопросах выбора вина.",
    input = "Привет! Какое вино посоветуете?"
)

printx(res.output_text)

Привет! 😊 Рад помочь с выбором вина! Чтобы подобрать идеальный вариант, расскажи немного подробнее:

1. **Ты предпочитаешь белое, красное, розовое или игристое вино?**  
2. **Какой вкус нравится — сухое, полусладкое, сладкое?**  
3. **Есть ли блюдо, к которому хочется подобрать вино (например, мясо, рыба, сыр, десерт)?**  
4. **На какое примерно бюджетное ограничение рассчитываем?**  
5. **Хочется что-то популярное и проверенное или, может, что-то необычное и интересное?**

С этими деталями я подберу отличный вариант! 🍷

Чтобы продолжить диалог, мы можем либо передать модели на вход всю историю диалога, либо указать ID предыдущего ответа, начиная с которого нужно продолжить диалог (при этом в предыдущем диалоге нам нужно указать `store=True`, чтобы переписка сохранялась):

In [28]:
print(f"ID предыдущего ответа: {res.id}")

res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    previous_response_id = res.id,
    input = "Я буду есть стейк!"
)

printx(res.output_text)

ID предыдущего ответа: a7f75359-e3e2-46d0-b9cc-72305c8546bf


Отлично! Стейк — это классика, и с ним отлично сочетаются насыщенные, структурированные вина. Вот мой подбор:

### 🍷 **Идеальные вина к стейку:**

#### 1. **Красное сухое вино — лучший выбор**
Стейк (особенно из говядины — рибай, филе, томахок) требует вина с хорошей танинностью, телом и глубоким вкусом. Танины "очищают" рот после жирного мяса, а насыщенный вкус вина дополняет умами-ноты мяса.

---

### 🔥 **Топ-рекомендации:**

#### 🌟 **Каберне Совиньон**  
— **Почему:** насыщенный, с танинами, чёрными ягодами, нотками чёрного перца, дуба, иногда — табака или графита.  
— **Где искать:** Нобель (Франция, Бордо), Калифорния (Напа Вэлли), Чили, Австралия.  
— **Пример:** *Château Larose Trintaudon (Bordeaux)* — элегантно, но мощно.

#### 🌟 **Мальбек**  
— **Почему:** сочный, фруктовый, с шелковистыми танинами и нотками чёрной вишни, шоколада, дыма.  
— **Где искать:** Аргентина (Мендоса) — мировой эталон!  
— **Пример:** *Catena Zapata Malbec* — баланс силы и изящества.

#### 🌟 **Сира/Шираз**  
— **Почему:** пряный, плотный, с нотами дикого мяса, перца, сливы.  
— **Где искать:** Сира — Франция (Северное Роно), Шираз — Австралия (Баросса Вэлли).  
— **Пример:** *Yalumba The Signature (Австралия)* — сложное, с глубоким послевкусием.

#### 🌟 **Неббиоло (например, Бароло или Барбареско)**  
— **Почему:** если хочется чего-то элитного. Высокие танины, кислотность, ароматы роз, трюфелей, кожи.  
— **Где искать:** Италия, Пьемонт.  
— **Пример:** *Vietti Barolo* — вино с характером!

---

### 💡 **Совет:**
Если стейк приготовлен на гриле или с дымком — выбирай вино с нотками дыма, дуба, копчёности (например, выдержанное в дубе Мальбек или Шираз).

---

### 🍷 А если хочется чего-то необычного?
Попробуй **Темпраниео из Рибера-дель-Дуэро (Испания)** — мощное, с тёмными фруктами и минеральностью, отлично ложится на стейк.

---

Хочешь, я подберу конкретную бутылку под твой бюджет? Напиши, сколько готов потратить — и будет точный совет! 🥩🍷

## Function Calling

Предположим, мы делаем ассистента для магазина вин, который торгует винами по некоторому прайс-листу. Возьмём пример такого прайс-листа:

In [6]:
import pandas as pd

pl = pd.read_excel("data/wine-price-ru.xlsx")
pl

,Id,Name,Country,Price,WHPrice,etc,Acidity,Color,Volume
0,56885,САССИКАЙЯ КР СХ,IT,27799.000,19459.3000,NaN,Сухое,Красное,0.750
1,666560,СИЕПИ МАЗЕЙ КР СХ,IT,15999.000,11199.3000,NaN,Сухое,Красное,0.750
2,533769,ПАЛАФРЕНО КР СХ,IT,14999.004,10499.3028,NaN,Сухое,Красное,0.750
3,93733,АНТ ТИНЬЯНЕЛЛО КР СХ,IT,14499.012,10149.3084,NaN,Сухое,Красное,0.750
4,644863,ШАТО МОНРОЗ КР СХ,FR,12999.000,9099.3000,от промо цены,Сухое,Красное,0.750
...,...,...,...,...,...,...,...,...,...
747,61418,КАГОР ТАМ КР СЛ,RU,179.004,125.3028,от промо цены,Сладкое,Красное,0.700
748,615581,ДЖАСТ МЕРЛО КР СХ,FR,149.004,104.3028,NaN,Сухое,Красное,0.187
749,615582,ДЖАСТ КБСВ КР СХ,FR,149.004,104.3028,NaN,Сухое,Красное,0.187
750,83302,АДАГУМ КБСВ КР СХ,RU,119.004,83.3028,NaN,Сухое,Красное,0.187


Предположим, мы хотим научиться отвечать на вопросы по прайс-листу, например, какое есть самое дешевое красное вино из Италии. Это можно сделать несколькими способами:

* Попытаться закинуть прайс-лист в контекст модели. Этот подход будет работать только для очень небольших таблиц. Конечно, можно попробовать использовать RAG (подробнее про это ниже), но без видения всей таблицы модель не сможет найти самое дешевое вино.
* Попытаться организовать трансляцию запроса не естественном языке в SQL-подобный язык. Это идеальный вариант, но его сложно сделать без ошибок без fine-tuning-а модели. 
* Извлечь из текстового запроса основные параметры того, что хочет пользователь, и затем сформировать на этой основе запрос, извлечающий данные из таблицы. Такой подход описан, например, в статье [Querying Databases with Function Calling](https://arxiv.org/html/2502.00032v1)

В любом случае, два последних варианта подразумевают использование технологии Function Calling - когда модель может сама решить, когда ей надо вызвать некоторую **внешнюю функцию**, и какие параметры нужно ей передать.

Чтобы function calling работал - нам надо сообщить LLM о доступных **инструментах**. Это можно сделать, передав с помощью JSON-схемы описание возможностей таких инструментов и их параметров.

Часто, чтобы не писать JSON-схему для function calling вручную, используют типизированные объекты Pyton pydantic. Для извлечения параметров запроса о вине, мы создадим такой объект: 

In [11]:
from pydantic import BaseModel, Field
from typing import Optional

class SearchWinePriceList(BaseModel):
    """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""

    name: str = Field(description="Название вина", default=None)
    country: str = Field(description="Страна на русском языке", default=None)
    acidity: str = Field(
        description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None
    )
    color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
    sort_order: str = Field(
        description="Порядок выдачи (most expensive, cheapest, random, average)",
        default=None,
    )
    what_to_return: str = Field(
        description="Что вернуть (wine info или price)", default=None
    )

Теперь создадим инструмент (tool) и передадим его в запрос модели. Также в инструкции ассистенту пропишем, что он может использовать Function Calling.

In [31]:
tools = [
    {
        "type": "function",
        "name": "Exercise",
        "description": "Вызывай, когда пользователь хочет получить информацию о конкретных винах или их стоимости.",
        "parameters": SearchWinePriceList.model_json_schema(),
    }
]

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если вопрос касается конкретных вин
или цены, то используй Function Calling.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

res = client.responses.create(
    model = model,
    store = True,
    tools = tools,
    instructions = instruction,
    input = "Какое самое дешевое вино из Австралии?"
)
res.to_dict()

{'id': '2905cfd1-ba6b-43fd-be68-a2a60be2c355',
 'created_at': 1758318017607.0,
 'error': None,
 'incomplete_details': None,
 'instructions': '\nТы - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина\nи рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. \nПосмотри на всю имеющуюся в твоем распоряжении информацию\nи выдай одну или несколько лучших рекомендаций. Если вопрос касается конкретных вин\nили цены, то используй Function Calling.\nЕсли что-то непонятно, то лучше уточни информацию у пользователя.\n',
 'metadata': None,
 'model': 'gpt://b1gst3c7cskk2big5fqn/qwen3-235b-a22b-fp8/latest',
 'object': 'response',
 'output': [{'arguments': '{"country": "Австралия", "sort_order": "cheapest", "what_to_return": "wine info"}',
   'call_id': 'chatcmpl-tool-e6b375abe35e444da6d28247ddeef072',
   'name': 'Exercise',
   'type': 'function_call',
   'id': 'chatcmpl-tool-e6b375abe35e444da6d28247ddeef072',
   'status': 'completed',
  

Видим, что в итоге вернулся ответ, указывающий, что надо сделать Funtion Call для получения данных из таблицы. При это LLM извлекла из текста все параметры запроса.

Реализуем функцию, которая возвращает список вин по параметрам, заданным в виде объекта `SearchWinePriceList`:

In [62]:
country_map = {
    "IT": "Италия",
    "FR": "Франция",
    "ES": "Испания",
    "RU": "Россия",
    "PT": "Португалия",
    "AR": "Армения",
    "CL": "Чили",
    "AU": "Австрия",
    "GE": "Грузия",
    "ZA": "ЮАР",
    "US": "США",
    "NZ": "Новая Зеландия",
    "DE": "Германия",
    "AT": "Австрия",
    "IL": "Израиль",
    "BG": "Болгария",
    "GR": "Греция",
    "AU": "Австралия",
}

revmap = {v.lower(): k for k, v in country_map.items()}


def find_wines(req):
    x = pl.copy()
    if req.country and req.country.lower() in revmap.keys():
        x = x[x["Country"] == revmap[req.country.lower()]]
    if req.acidity:
        x = x[x["Acidity"] == req.acidity.capitalize()]
    if req.color:
        x = x[x["Color"] == req.color.capitalize()]
    if req.name:
        x = x[x["Name"].apply(lambda x: req.name.lower() in x.lower())]
    if req.sort_order and len(x)>0:
        if req.sort_order == "cheapest":
            x = x.sort_values(by="Price")
        elif req.sort_order == "most expensive":
            x = x.sort_values(by="Price", ascending=False)
        else:
            pass
    if x is None or len(x) == 0:
        return "Подходящих вин не найдено"
    return "Вот какие вина были найдены:\n" + "\n".join(
        [
            f"{z['Name']} ({country_map.get(z['Country'],'Неизвестно')}) - {z['Price']}"
            for _, z in x.head(10).iterrows()
        ]
    )


print(find_wines(SearchWinePriceList(country="Австралия", sort_order="cheapest")))

Вот какие вина были найдены:
 ДЖИНДАЛИ КБСВ КР ПСХ (Австралия) - 499.0
 ДЖИНДАЛИ МЕРЛО КР ПСХ (Австралия) - 499.0
 ЧОЛК ХИЛЛ ШИРАЗ КР СХ (Австралия) - 509.0
 ПИТ'С ПЮР ПННР КР ПСХ (Австралия) - 579.0
 ПИТ'С ПЮР ШИРАЗ КР ПСХ (Австралия) - 579.0
 СТАМП ДЖАМП КР СХ (Австралия) - 789.0
 ЛИНД БИН50 ШИР КР ПСХ (Австралия) - 899.0
 ЛЭКИ ШИРАЗ КРСХ (Австралия) - 978.996
 СТЭДФАСТ ШИР КАБ КРСХ (Австралия) - 999.0
 ТИРРЕЛЗ ШИР КР СХ (Австралия) - 1098.996


В ответ на Function Call нам нужно сформировать ответ от функции, и передать её обратно для обработки языковой моделью. В нашем случае функция может быть только одна, поэтому мы не проверяем название функции, а всегда запрашиваем поиск в таблице:

In [33]:
import json 

tool_calls = [item for item in res.output if item.type == "function_call"]

if tool_calls:
    out = []
    for call in tool_calls:
        print(f" + Обрабатываем: {call.name} (call_id={call.call_id}, args={call.arguments})")
        try:
            args = json.loads(call.arguments)
            args = SearchWinePriceList.model_validate(args)
            result = find_wines(args)
        except Exception as e:
            result = f"Ошибка: {e}"
        print(f" + Результат: {result}")
        out.append({
            "type": "function_call_output",
            "call_id": call.call_id,
            "output": result
        })
        res = client.responses.create(
            model=model,
            input=out,
            tools=tools,
            previous_response_id=res.id,
            store=True
        )

printx(res.output_text)

 + Обрабатываем: Exercise (call_id=chatcmpl-tool-e6b375abe35e444da6d28247ddeef072, args={"country": "Австралия", "sort_order": "cheapest", "what_to_return": "wine info"})
 + Результат: Вот какие вина были найдены:
 ДЖИНДАЛИ КБСВ КР ПСХ (Австралия) - 499.0
 ДЖИНДАЛИ МЕРЛО КР ПСХ (Австралия) - 499.0
 ЧОЛК ХИЛЛ ШИРАЗ КР СХ (Австралия) - 509.0
 ПИТ'С ПЮР ПННР КР ПСХ (Австралия) - 579.0
 ПИТ'С ПЮР ШИРАЗ КР ПСХ (Австралия) - 579.0
 СТАМП ДЖАМП КР СХ (Австралия) - 789.0
 ЛИНД БИН50 ШИР КР ПСХ (Австралия) - 899.0
 ЛЭКИ ШИРАЗ КРСХ (Австралия) - 978.996
 СТЭДФАСТ ШИР КАБ КРСХ (Австралия) - 999.0
 ТИРРЕЛЗ ШИР КР СХ (Австралия) - 1098.996


Самые дешёвые вина из Австралии стоят **499 рублей**. Это:

- **ДЖИНДАЛИ КБСВ КР ПСХ** (красное, полусухое)
- **ДЖИНДАЛИ МЕРЛО КР ПСХ** (красное, полусухое)

Таким образом, для вызова функции необходимо:
* Сообщить модели о доступных функциях
* При вызове модели обработать функциональный вызов, если в результате вызова модель вернула соответствующий ответ

## Релизуем агента с Function Calling

Для реализации полноценного ассистента добавим ещё функцию добавления вина в корзину, распечатку корзины и передачи общения оператору.

Немного структурируем наш код:
* Добавим функцию для обработки вызова фукции прямо в класс с описанием данных
* Создадим класс `Assistant`, который будет реализовывать функциональный вызов, а также автоматически поддерживать диалог, запоминая идентификаторы предыдущих ответов.

> Чтобы ассистент мог поддерживать диалог сразу с несколькими пользователями, нам нужно также ввести некоторый идентификатор сессии `session_id`, и для каждой сессии помнить свою историю переписки и `id` последнего сообщения. Идентификатор сессии мы также будет передавать во все функции `session_id`.


In [63]:
class SearchWinePriceList(BaseModel):
    """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""

    name: str = Field(description="Название вина", default=None)
    country: str = Field(description="Страна на русском языке", default=None)
    acidity: str = Field(
        description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None
    )
    color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
    sort_order: str = Field(
        description="Порядок выдачи (most expensive, cheapest, random, average)",
        default=None,
    )
    what_to_return: str = Field(
        description="Что вернуть (wine info или price)", default=None
    )

    def process(self, session_id):
        return find_wines(self)

Также в виде функции реализуем функциональность передачи управления оператору. В данном случае мы будем просто устанавливать некоторый глобальный флаг `handover` - это очень грубое решение и сделано для простоты, в реальной жизни надо будет подумать над более правильной логикой, поддерживающей нескольких пользователей.

In [64]:
handover = False

class Handover(BaseModel):
    """Эта функция позволяет передать диалог человеку-оператору поддержки"""

    reason: str = Field(
        description="Причина для вызова оператора", default="не указана"
    )

    def process(self, session_id):
        global handover
        handover = True
        return f"Я побежала вызывать оператора, ваш {session_id=}, причина: {self.reason}"

Также реализуем функцию добавления вин в корзину. Чтобы для каждого пользователя была своя корзина, будем привязывать её к идентификатору сессии:

In [65]:
carts = {}

class AddToCart(BaseModel):
    """Эта функция позволяет положить или добавить вино в корзину"""

    wine_name: str = Field(
        description="Точное название вина, чтобы положить в корзину", default=None
    )
    count: int = Field(
        description="Количество бутылок вина, которое нужно положить в корзину",
        default=1,
    )

    def process(self, session_id):
        if session_id not in carts:
            carts[session_id] = []
        carts[session_id].append(self)
        return f"Вино {self.wine_name} добавлено в корзину, число бутылок: {self.count}"

Наконец, оформим функцию для показа корзины:

In [66]:
class ShowCart(BaseModel):
    """Эта функция позволяет показать содержимое корзины"""

    def process(self, session_id):
        if session_id not in carts or len(carts[session_id]) == 0:
            return "Корзина пуста"
        return "В корзине находятся следующие вина:\n" + "\n".join(
            [f"{x.wine_name}, число бутылок: {x.count}" for x in carts[session_id]]
        )

Теперь реализуем главный класс `Agent`, который будет брать на себя обработку функций. В качестве `tools` будем передавать список описанных нами ранее классов.

In [67]:
class Agent():
    user_sessions = {}

    def __init__(self, instruction, tools = [], session_id='default', model=model):
        self.instruction = instruction
        self.model = model
        self.tool_map = { x.__name__ : x for x in tools if issubclass(x, BaseModel) }
        self.tools = [
            self._create_tool_annot(x) for x in tools
        ]
        if session_id not in self.user_sessions:
            self.user_sessions[session_id] = {
                "last_reply_id" : None,
                "history" : [],
            }

    def _create_tool_annot(self, x):
        if issubclass(x, BaseModel):
            return {
                "type": "function",
                "name": x.__name__,
                "description": x.__doc__,
                "parameters": x.model_json_schema(),
            }
        else:
            return x

    def __call__(self, message, session_id='default'):
        s = self.user_sessions[session_id]
        s['history'].append({ 'role': 'user', 'content': message })
        res = client.responses.create(
            model = self.model,
            store = True,
            tools = self.tools,
            instructions = self.instruction,
            input = message
        )
        # Обрабатываем вызов локальных инструментов
        tool_calls = [item for item in res.output if item.type == "function_call"]
        if tool_calls:
            s['history'].append({ 'role' : 'func_call', 'content' : res.output_text })
            out = []
            for call in tool_calls:
                print(f" + Обрабатываем: {call.name} ({call.arguments})")
                try:
                    fn = self.tool_map[call.name]
                    obj = fn.model_validate(json.loads(call.arguments))
                    result = obj.process(session_id)
                except Exception as e:
                    result = f"Ошибка: {e}"
                print(f" + Результат: {result}")
                out.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
                res = client.responses.create(
                    model=self.model,
                    input=out,
                    tools=self.tools,
                    previous_response_id=res.id,
                    store=True
                )
        self.user_sessions[session_id]['last_reply_id'] = res.id
        s['history'].append({ 'role' : 'assistant', 'content' : res.output_text })
        return res

    def history(self, session_id='default'):
        return self.user_sessions[session_id]['history']

Создадим нашего агента - винного сомелье. В системном промпте пропишем ему возможные варианты для вызова функции, чтобы облегчить задачу.

In [9]:
instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Если вопрос касается конкретных вин или цены, то вызови функцию SearchWinePriceList.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[SearchWinePriceList, Handover, AddToCart, ShowCart],
)

NameError: name 'SearchWinePriceList' is not defined

In [40]:
printx(wine_agent("Какое вино пьют со стейком?").output_text)

Со стейком обычно пьют красные вина, особенно выдержанные и насыщенные сорта, такие как Каберне Совиньон, Мерло или Сира (Шираз). Они хорошо сочетаются с жирным мясом благодаря своей структуре и танинам.

Хотите, чтобы я подобрал конкретные вина из прайс-листа, подходящие к стейку? Можно уточнить предпочтения — страну, ценовой диапазон или уровень выдержки.

In [41]:
printx(wine_agent("Какие вина Кьянти есть в продаже?").output_text)

 + Обрабатываем: SearchWinePriceList ({"name": "Кьянти", "what_to_return": "wine info"})
 + Результат: Вот какие вина были найдены:
 КВЕРЧАБ КЬЯНТИ КР СХ (Италия) - 2499.0
 ПОЛИЦ КЬЯНТИ КР СХ (Италия) - 1749.756
 КАСАЛ КЬЯНТИ СУП КРСХ (Италия) - 1349.004
 ВЕК КАНТ КЬЯНТИ КР СХ (Италия) - 1099.0
 ПРЕДЕЛЛА КЬЯНТИ КРСХ (Италия) - 999.0
 ЗОНИН КЬЯНТИ КР СХ (Италия) - 699.0
 ПРЕДЕЛЛА КЬЯНТИ КРСХ (Италия) - 369.0


Вот список вин Кьянти, доступных в продаже:

1. **КВЕРЧАБ КЬЯНТИ КР СХ (Италия)** — 2499.0 руб.
2. **ПОЛИЦ КЬЯНТИ КР СХ (Италия)** — 1749.76 руб.
3. **КАСАЛ КЬЯНТИ СУП КРСХ (Италия)** — 1349.00 руб.
4. **ВЕК КАНТ КЬЯНТИ КР СХ (Италия)** — 1099.0 руб.
5. **ПРЕДЕЛЛА КЬЯНТИ КРСХ (Италия)** — 999.0 руб.
6. **ЗОНИН КЬЯНТИ КР СХ (Италия)** — 699.0 руб.
7. **ПРЕДЕЛЛА КЬЯНТИ КРСХ (Италия)** — 369.0 руб.

Если хотите, я могу помочь добавить любое из них в корзину.

In [42]:
printx(wine_agent("Добавь в корзину Полиц Кьянти, три бутылки").output_text)

 + Обрабатываем: AddToCart ({"wine_name": "Полиц Кьянти", "count": 3})
 + Результат: Вино Полиц Кьянти добавлено в корзину, число бутылок: 3


Вино "Полиц Кьянти" успешно добавлено в корзину — 3 бутылки. Хотите посмотреть содержимое корзины или продолжить покупки?

In [43]:
printx(wine_agent("Ещё положи в корзину Зонин Кьянти").output_text)

 + Обрабатываем: AddToCart ({"wine_name": "Зонин Кьянти", "count": 1})
 + Результат: Вино Зонин Кьянти добавлено в корзину, число бутылок: 1


Вино "Зонин Кьянти" успешно добавлено в корзину! Всего одна бутылка. Хотите что-то ещё или готовы оформить заказ?

In [44]:
printx(wine_agent("Что у меня в корзине?").output_text)

 + Обрабатываем: ShowCart ({})
 + Результат: В корзине находятся следующие вина:
Полиц Кьянти, число бутылок: 3
Зонин Кьянти, число бутылок: 1


В вашей корзине находятся:

- **Полиц Кьянти** — 3 бутылки  
- **Зонин Кьянти** — 1 бутылка  

Если хотите, я могу помочь добавить ещё вина или оформить заказ.

In [45]:
printx(wine_agent("Вызови оператора, хочу оформить доставку!").output_text)

 + Обрабатываем: Handover ({"reason": "Оформление доставки"})
 + Результат: Я побежала вызывать оператора, ваш session_id='default', причина: Оформление доставки


Оператор уже в пути! Он поможет вам с оформлением доставки. Ожидайте, пожалуйста.

## Model Context Protocol

Мы научились интегрировать в ассистента локальные функции, но иногда бывает так, что функции предоставляются какими-то внешними поставщиками информации. Предположим, мы хотим сделать универсального ассистента, который сможет анализировать цены в различных интернет-магазинах. Для этого нужно, чтобы интернет-магазины предоставляли доступ к своим прайс-листам в виде удалённых функций, доступных через протокол MCP: Model Context Protocol. 

Реализуем такой MCP-сервер для доступа к прайс-листу магазина. Для этого мы используем библиотеку [FastMCP](https://gofastmcp.com/). Пример реализации сервера доступен [здесь](mcp-server/mcp-wine-shop.py), запустить его можно командой
```bash
fastmcp run mcp-wine-shop.py -t sse -p 3000 --host 0.0.0.0
```

Вы также можете запустить сервер с помощью [FastMCP Cloud](https://fastmcp.cloud/) используя [этот GitHub-репозиторий](https://github.com/yandex-datasphere/advanced-assistant-mcp)

In [40]:
mcp_tool = {
            "type": "mcp",
            "server_label": "Wine-Shop",
            "server_description": "Функция для запроса цен на вино в винном магазине",
            #"server_url": "https://wineparadise.fastmcp.app/mcp",
            "server_url": "http://cathy.ycloud.eazify.net:8000/sse",
            "require_approval": "never",
    }

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[mcp_tool],
)

In [41]:
printx(wine_agent("Сколько стоит самый дорогой Мерло?").output_text)

Самый дорогой Мерло стоит 2449.656 рублей. Это вино "ЛЕФКАДИЯ МЕРЛО КР СХ" из России.

## Добавляем RAG

Наш ассистент сам по себе неплохо отвечает на вопросы по винам, однако если мы хотим сделать его знания более предсказуемыми - необходимо дополнить их специализированной информацией из текстовой базы знаний. Для этого мы можем использовать RAG - подход.

В качестве текстовой базы знаний в нашем примере возьмём тексты о винах из энциклопедии: про разные сорта винограда и про регионы произрастания:


In [57]:
from glob import glob
import pandas as pd
import tiktoken

def get_tokenizer(model):
    try:
        return tiktoken.encoding_for_model(model)
    except:
        return tiktoken.encoding_for_model("gpt-5-nano") 

def get_token_count(filename):
    tokenizer = get_tokenizer(model)
    with open(filename, "r", encoding="utf8") as f:
        return len(tokenizer.encode(f.read()))

def get_file_len(filename):
    with open(filename, encoding="utf-8") as f:
        l = len(f.read())
    return l

d = [
    {
        "File": fn,
        "Tokens": get_token_count(fn),
        "Chars": get_file_len(fn),
    }
    for fn in list(glob("data/wines.txt/*.txt"))+list(glob("data/regions.txt/*.txt"))
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars
0,data/wines.txt\Альбариньо.txt,763,2387
1,data/wines.txt\Блауфранкиш.txt,787,2596
2,data/wines.txt\Вионье.txt,651,2117
3,data/wines.txt\Виура.txt,324,1038
4,data/wines.txt\Гевюрцтраминер.txt,736,2236
...,...,...,...
125,data/regions.txt\Штирия.txt,619,2078
126,data/regions.txt\Элгин.txt,346,1081
127,data/regions.txt\Элим.txt,346,1110
128,data/regions.txt\Эльзас.txt,530,1887


Чтобы агент мог обращаться к этим файлам - загружаем их в облако:

In [58]:
from tqdm.auto import tqdm
tqdm.pandas()

def upload_file(filename):
    return client.files.create(
        file=open(filename,'rb'),
        purpose='assistants')

df["Uploaded"] = df["File"].progress_apply(upload_file)

100%|██████████| 130/130 [00:27<00:00,  4.65it/s]


Поместим все загруженные файлы в векторную базу данных. Если файлы достаточно большие, то в процессе индексации можно применить **чанкование** - нарезку файлов на перекрывающиеся фрагменты с заданной длинной и перекрытием (в токенах). Мы зададим эти параметры, но поскольку в нашем случае текстовые файлы невелики - чанкование не потребуется.

In [59]:
vector_store = client.vector_stores.create(name='rag_store')

def add_to_store(file):
    client.vector_stores.files.create(
        vector_store_id=vector_store.id, 
        file_id=file.id,
        chunking_strategy={
            "type": "static",
            "static" : { "max_chunk_size_tokens" : 1000, "chunk_overlap_tokens" : 100 }
        }
        )

_ = df['Uploaded'].progress_apply(add_to_store)

100%|██████████| 130/130 [01:32<00:00,  1.40it/s]


Теперь мы можем в явном виде искать фрагменты в нашем векторном хранилище:

In [60]:
res = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="Какого цвета вино Зинфандель?"
)
for x in res.data:
    print(f"{len(x.content[0].text)} символов из файла {x.filename}, релевантность = {x.score}")

844 символов из файла Зинфандель.txt, релевантность = 0.999999996975375
834 символов из файла Зинфандель.txt, релевантность = 0.4837495206124824
459 символов из файла Калифорния.txt, релевантность = 0.45836024800124275
876 символов из файла Зинфандель.txt, релевантность = 0.4474486988545032
263 символов из файла Орегон.txt, релевантность = 0.33309060260960344
2599 символов из файла Блауфранкиш.txt, релевантность = 0.15973148502296913
971 символов из файла Калифорния.txt, релевантность = 0.11312163164976587
956 символов из файла Орегон.txt, релевантность = 0.11312163164976587
948 символов из файла Вионье.txt, релевантность = 0.09895006851783038
1507 символов из файла Жюрансон.txt, релевантность = 0.08397052371146162



Теперь собираем собственно ассистента, который будет использовать RAG. Для этого определяем **инструмент** (tool) для поиска в нашем индексе, и указываем его при создании ассистента. Также важно задать хорошую инструкцию для ассистента (системный промпт): 

In [68]:
search_tool = {
    "type" : "file_search",
    "vector_store_ids" : [vector_store.id]
}

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера Wine-Shop.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[mcp_tool,search_tool,AddToCart,Handover],
)

In [69]:
res = wine_agent("Какое вино подходит к стейку?")
printx(res.output_text)

Для стейка лучше всего подходят красные вина с хорошей структурой и танинами, например, Каберне Совиньон, Мальбек или Сира (Шираз). Они хорошо сочетаются с насыщенным вкусом мяса.

Хотите, я подберу конкретные вина, подходящие к стейку? Можно уточнить, какое именно мясо (говядина, баранина и т.д.) и как вы его готовите (с кровью, средней прожарки и т.п.), а также ваш бюджет?

Посмотрим, из каких источников был получен этот ответ.

In [ ]:
from openai.types.responses.response_output_message import ResponseOutputMessage

def find_obj(t,l):
    for x in l:
        if isinstance(x,t):
            return x
    return None

def print_citations(result):
    o = find_obj(ResponseOutputMessage,result.output)
    for x in o.content[0].annotations:
        if x.type == "file_citation":
            print(f"{x.filename}, idx={x.index}")

print_citations(res)

Мы видим, что ответ получился несколько однобоким, поскольку данные о сочетании вин и еды содержатся в текстовой базе знаний в разрозненном виде.

## Добавляем таблицу соответствий

Поскольку подбор блюда к вину является частой задачей, добавим к нашей базе знаний явную табличку соответствий блюд и вин, которая находится в файле `data/food_wine_table.md` в формате markdown.

In [72]:
with open("data/food_wine_table.md", encoding="utf-8") as f:
    food_wine = f.readlines()
fw = "".join(food_wine)

tokenizer = get_tokenizer(model)
tokens = len(tokenizer.encode(fw))
print(f"Токенов: {tokens}, {len(fw)/tokens} chars/token")

Токенов: 15803, 2.6457001835094602 chars/token


In [73]:
printx(fw[:1000])

Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Баклажаны, запеченые с сыром | Красное вино: «среднетелые»* сухие — Гренаш (Гарнача), Санджовезе (Кьянти), Карменер, Менсия, молодые Темпранильо, легкотелое Мерло.
Баранина деликатесная (филе или каре ягненка) | Красное вино: сухие выдержанные вина из винограда Пино Нуар, Менсия, Неббиоло (в том числе элегантные выдержанные Бароло и Барбареско), Гамэ (элегантные бургундские Божоле Виляж).
Баранина пикантная: жареная, гриль, тушеная — со специями | Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России.
Бефстроганов | Белые вина: выдержанные в дубе Шардоне, Пино Гриджо (лучше — из Северной Италии), Вердехо, Вермент

Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [74]:
header = food_wine[:2]
header

['Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду\n',
 '--------|--------\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [75]:
import io

chunk_size = 600 * 5  # около 600 tokens * 5 char/token

s = header.copy()
uploaded_chunks = []
i = 0
for x in food_wine[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        f = client.files.create(
            purpose="assistants",
            file = (f'table_{i}.txt',io.BytesIO("".join(s).encode("utf-8")),'text/markdown')
        )
        client.vector_stores.files.create(file_id=f.id, vector_store_id=vector_store.id)
        uploaded_chunks.append(f)
        i+=1
        s = header.copy()
print(f"Uploaded {len(uploaded_chunks)} table chunks")

Uploaded 13 table chunks


Посмотрим, откуда теперь агент берёт информацию о соответствиях:

In [96]:
res = wine_agent("Какое вино подходит к стейку?")
printx(res.output_text)
print_citations(res)

К стейку чаще всего подают красные вина с хорошей структурой и танинами, которые помогают пережёвывать жир и мясной вкус. Ниже — практичные варианты в зависимости от способа приготовления и прожарки.

Рекомендации по прожарке Rare (редкая прожарка, жирный стейк, например рибай)
- Темпранильо из Риуэры дель Дуэро (выложенные и выдержанные, Crianza/Reserva) — классический мощный выбор с хорошо развитым букетом и терпкими танинами.
- Санджовезе Ризерва (Кьянти Ризерва) или Брунелло ди Монтальчино — элегантные, насыщенные вина с нужной структурой.
- Супертосканы (Blend из каже Каберне/Сира/Мальбек и т. п.) — глубокие, гладкие и долгоиграющие.
- Бордо Правого берега (мералджа/мерло в смеси) — плотные и благородные, отлично сочетаются с мясом.
- Аргентианские Мальбеки — бархатные и насыщенные, хорошо «обхватывают» жир стейка.
Итог: для Rare выбирайте мощные, со зрелыми танинами и хорошей выдержкой варианты из Tempranillo, Sangiovese Riserva, Супертосканы, Бордо правого берега и Мальбек. 

Рекомендации по прожарке Medium и Well-Done (более прожаренное мясо, требует чуть меньшей жесткости танинов или большего обогащения вкуса)
- Сира (Шираз) — интенсивная, пряная и мощная, хорошо переносит более плотную корочку.
- Каберне Совиньон — классический плотный выбор с хорошей структурой и долгим послевкусием.
- Мальбек (крупноплотный, «тельный» стиль) — хорошо совместим с прожаркой Medium и выше.
- Примитиво и Зинфандель — полнокровные, сладковато-фруктовые варианты, которые приятно контрастируют с жареным мясом.
- Альянико выдержанное — бархатное и с характером, подходит для более сложных стейков и маринадов.
- Тиражированные «ронские» ассамбляжи Гренаш+Сира+Мурведр — комплексные вина, хорошо держат тяжёлый вкус жареного мяса.
Итог: для Medium/Well-Done выбирайте Сира, Каберне Совиньон, Мальбек или их коктейли/ассамбляжи; можно рассмотреть Примитиво, Зинфандель и выдержанные Альянико. 

Дополнительно (варианты по вкусу)
- Вина из Сицилии, например нерелло маскалезе, могут отлично дополнять мясные блюда за счёт ярких танинов и минеральности, особенно если стейк подрумянен на гриле. 
- Есть и другие региональные примеры в вашем наборе таблиц: например, сочетания для стейков и жирной говядины встречаются и в разных локализациях (таблицы с мясными блюдами и стейками). Если хотите, могу привести конкретные названия из вашего региона и ценовые варианты. 

Как сделать подбор точнее
- Расскажите, какой у вас стейк (типа — рибай, филе, стрип), какая прожарка, есть ли соусы ( pepper, грибной соус, барбекю) и какой бюджет.
- Скажите, предпочитаете ли вы конкретные регионы или сорта (например, только Tempranillo или только Каберне Совиньон).

Готов подобрать конкретные названия и цены под ваш бюджет и регион. Хотите, чтобы я нашёл подходящие варианты в вашем магазине Wine-Shop и добавил пару бутылок в корзину? Если да, скажите бюджет и желаемые регионы, и я запрошу цены и availability.

table_10.md, idx=996
table_10.md, idx=1904
Сицилия.md, idx=2104
table_10.md, idx=2376


## Многоагентное тестирование

Когда мы сделали такого бота, возникает вопрос, как его тестировать. Для этого возможно несколько вариантов:

* Ручное тестирование (примерно то, что мы проделали выше)
* Автоматическое тестирование на заранее заготовленном датасете диалогов, с формализованной проверкой метрик. Такое тестирование удобно проводить с помощью специализированных фреймворков, например, RAGAS.

Мы в данном примере предлагаем промежуточный по сложности подход - это **многоагентное тестирование**, при котором мы инструктируем другого агента, чтобы он побеседовал с нашим винным ассистентом в определённом ключе. В итоге у нас могут получиться различные сценарии диалога, глядя на которые можно будет определить направление доработки исходного ассистента.

In [98]:
instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера Wine-Shop.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[mcp_tool,search_tool,AddToCart,Handover],
)

In [99]:
instruction_user = """
Ты - простой человек, и тебе нужно выбрать вино в интернет-магазине для ужина.
Ты не очень много понимаешь в винах, и хочешь побольше распросить сомелье о разных вариантах.
Ты хотел бы съесть стейк, но готов изменить свой выбор еды, если поймёшь хорошее сочетание с вином. 
Говори простым языком, короткими разговорными фразами. Когда ты удовлетворём выбором, попроси 
найти самое недорогое вино данного типа, затем положи его в корзину. В конце попроси соединить тебя 
с оператором, чтобы оформить доставку. Каждое действие выполняй в одной фразой в ответ на сообщение сомелье. Никогда 
не продолжай диалог, больше, чем одной фразой. Не пиши реплики от лица сомелье или кого-то другого.
"""

user = Agent(instruction=instruction_user)

msg = "Добрый день! Поможете мне выбрать вино?"
handover = False
for i in range(10):
    printx(f"**Посетитель:** {msg}")
    msg = wine_agent(msg).output_text
    printx(f"**Сомелье:** {msg}")
    if handover:
        break
    msg = user(msg).output_text

**Посетитель:** Добрый день! Поможете мне выбрать вино?

**Сомелье:** Здравствуй! Конечно, помогу выбрать. Чтобы подобрать наиболее подходящее вино, скажи, пожалуйста:
- какое блюдо ты планируешь подать (или какой стиль крема/сала/мясо)?
- предпочитаешь белое, красное или розовое?
- примерный бюджет?

Пока ждем уточнения, вот 3 готовых сочетания из наших справочников, которые подойдут в разных случаях:

1) Рыба и морепродукты (легкое, освежающее белое)
- варианты: тихие белые вина с хорошей кислотностью — Пино Гри, Пино Блан, Виура, Рислинг; можно рассмотреть и Рислинг в стиль Старого Света или Совиньон Блан с характерной свежестью. Также к морепродуктам хорошо идут элегантные белые игристые.
- примеры из наших материалов: для лёгкой рыбы и морепродуктов рекомендуется белое тихое с лёгким телом (Пино Гри, Пино Блан, Виура) и Рислинг; к устрицам — кислотоносные белые вроде Совиньон Блан и Рислинг; к блюдам в азиатском стиле — яркие Гевюрцтраминер и Мускат. Подробности можно увидеть в описаниях к таким блюдам как рыба и устричные пары в таблицах.  

2) Мясные блюда и стейки (красное среднетелое)
- варианты: Неббиоло, Пино Нуар, Менсия, Санджовезе, Мерло — под строгие, но не слишком тяжёлые мясные блюда. Для мясных рагу и стейков часто рекомендуют «тельные» стили. 
- примеры из материалов: для блюд вроде стейка и жареного лука подойдут сухие/полусухие вина из Гарнача/Гренаш, Мерло, Карменер, Менсия, а также тельные Пино Нуары и Неббиоло.  Также встречаются рекомендации по Неббило и Пино Нуар в сочетаниях с мясными блюдами. 

3) Паста/пицца и итальянские блюда (классика — красное, иногда бланко-окрашенное)
- варианты: Санджовезе (Кьянти), Неро д’Авола, Барбера, Монтералло/Мерло, Пино Нуар — в зависимости от блюда. Для более лёгких пицц подойдут «легкие» красные, для мясных — более тельные. Также встречаются сочетания с Оранжи и розовыми винами.
- примеры из материалов: пицца и паста часто сочетаются с Санджовезе, Неро д’Авола, Неббиоло, Гамэ; для пиццы Маргарита — 추천 красные вина: Санджовезе, Неро д’Авола, Негроамаро, Барбера, Темпранильо.  Для лазаньи и рагу — Санджовезе, Неро д’Авола, Барбера, Неббиоло и т.д. 

Хочешь, могу прямо сейчас подобрать конкретные бутылки и показать цену в магазине или добавить выбранное к корзине. Сообщи, пожалуйста, какой из вариантов тебе ближе, какие блюда ты планируешь и примерно сколько хочешь потратить.

**Посетитель:** Уточни, пожалуйста: какое блюдо планируешь (или стиль блюда), красное/белое/розовое и какой примерный бюджет на бутылку?

**Сомелье:** Отлично, давайте сузим выбор. Ответьте, пожалуйста, на три вопроса:

1) Что за блюдо планируете или какой стиль кухни? (пример: стейк из говядины, паста с грибами, морепродукты, блюдо из курицы, сырная тарелка, лёгкий салат и т.д.)

2) Какой цвет вина предпочитаете: Красное / Белое / Розовое (или неважно)?

3) Бюджет на бутылку в рублях (примерные диапазоны: до 1500 / 1500–3000 / 3000–5000 / 5000–10000 / 10000+).

После ваших ответов могу предложить 3–4 конкретных варианта и при необходимости проверить их в MCP-Wine-Shop. Если хотите, можно сразу сформулировать запрос типа: “к стейку красное до 4000 руб”.

**Посетитель:** 1) Блюдо: стейк из говядины; 2) Красное; 3) До 4000 руб.

**Сомелье:** Отличный выбор сочетания: красное к стейку. Ниже несколько вариантов из магазина в диапазоне до 4000 ₽, которые обычно хорошо работают с мясом и имеют сухую стиль:

- Адáгум Каберне Совиньон, Красное, сухое, Россия — около 119 ₽
  Прозрачный, ягоды черной смородины, крепкие танины, аккуратная структура. Хорошо сочетается с жареным стейком и румяной корочкой.

- Джаст Каберне Совиньон, Красное, сухое, Франция — около 149 ₽
  Классический каберне с выразимой ягодной палитрой и заметной структурой. Подкрепит вкус мяса без перегруженности.

- Джаст Мерло, Красное, сухое, Франция — около 149 ₽
  Более мягкий стиль по сравнению с каберне, бархатистый вкус. Хороший выбор, если хотите менее агрессивные танны к стейку.

- Авторское №1, Красное, сухое, Россия — около 119 ₽
  Доступное сухое красное с понятной танинной структурой; подойдет для тех, кто любит простые и яркие вина к жаркому.

Примечание по выбору:
- Если хотите более «плотный» и танинный стиль к жирному бифштексу, выбирайте каберне (Адaгум или Джаст Каберне).
- Если предпочитаете чуть более мягкое сопровождение, можно выбрать мерло (Джаст Мерло).

Хотите, добавлю один или несколько из них в корзину? Или желаете увидеть больше вариантов в рамках до 4000 ₽ (например, более насыщенные или из других стран)?

**Посетитель:** Хочу добавить один из них в корзину, или показать больше вариантов до 4000 ₽ (например, более насыщенные или из других стран)?

 + Обрабатываем: Handover ({"reason":"не указана"})
 + Результат: Я побежала вызывать оператора, ваш session_id='default', причина: не указана


**Сомелье:** 

## Делаем ассистента проактивным

Мы обычно воспринимаем ассистентов как вопрос-ответных ботов, которые способны отвечать на вопросы пользователей. Однако с помощью системного промпта мы можем добавить им проактивности, чтобы они проявляли инициативу и сами инициировали диалог. Например:

In [103]:
instruction = """
Ты - опытный сомелье, продающий вино в магазине. Твоя задача - отвечать на вопросы пользователя
про вина, рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина,
а также проактивно предлагать пользователю приобрести вина, отвечающие его потребностям. В ответ
на сообщение /start поинтересуйся, что нужно пользователю, предложи ему какой-то
интересный вариант сочетания еды и вине, и попытайся продать ему вино.
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера Wine-Shop.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке.
Если что-то непонятно, то лучше уточни информацию у пользователя. Общайся достаточно короткими 
разговорными фразами, не используй перечисления, списки, длинные выдержки текста.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[search_tool, mcp_tool, Handover, AddToCart, ShowCart],
)

In [104]:
printx(wine_agent('/start').output_text)

Привет! Чем могу помочь? Что ищете — стиль вина, бюджет, к какому блюду?

Вот интересная пара сочетаний и несколько вариантов из нашего прайс-листа:
- Sassicaia Красное сухое, Италия — 27 799 ₽. К стейку, рагу или выдержанному мясу — классика, прямо как на дегустации.
- СИЕПИ МАЗЕЙ Красное сухое — 15 999 ₽. Хорошо к пасте с мясным соусом или жареному цыпленку.
- Палефрено Красное сухое — 14 999 ₽. Отличный повседневный выбор к пасте, запеканкам и мясным блюдам.

Если хотите, могу подобрать под блюдо конкретное сочетание или добавить одну бутылку в корзину. Что будете готовить или какой бюджет предпочитаете?

In [105]:
printx(wine_agent('Сколько стоит вино под стейк?').output_text)

Для стейка подойдут красные вина с выразительными танинами. В нашем прайс-листе есть такие варианты и их цены:

АВТОРСКОЕ №1 КР СХ (Россия) — 119 ₽; АДАГУМ КБСВ КР СХ (Россия) — 119 ₽; ДЖАСТ КБСВ КР СХ (Франция) — 149 ₽; ДЖАСТ МЕРЛО КР СХ (Франция) — 149 ₽; КАГОР ТАМ КР СЛ (Россия) — 179 ₽.

Мой выбор по сочетанию цена-качество — ДЖАСТ КБСВ КР СХ (Франция) за 149 ₽: крепкий, танинами держит мясо и хорошо подходит к стейку с жареным картофелем.

Хочешь, добавлю одну бутылку в корзину или подобрать что-то под твой бюджет?

## Делаем винного ассистента в телеграме

Для демонстрации возможностей бота мы реализуем такого ассистента в виде телеграм-бота. Конечно, для реализации полноценного телеграм-бота необходимо использовать виртуальную машину и режим webhooks, но в нашем случае мы ограничимся режимом поллинга, и запустим бота прямо в Datasphere.

> Прежде, чем запускать код ниже, необходимо создать чат-бота, пообщавшись с [@botfather](http://t.me/botfather), и разместить его секрет в виде секрета в Datasphere.

Для начала установим необходимую библиотеку:

In [89]:
%pip install --quiet telebot

I0000 00:00:1743586576.365735    7698 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import telebot

telegram_token = os.environ["tg_token"]

bot = telebot.TeleBot(telegram_token)

sessions = {}

# Обработчик команды /start
@bot.message_handler(commands=["start"])
def start(message):
    session_id = message.chat.id
    print(f"Starting on session {session_id}, msg={message.text}")
    ans = wine_agent(message.text, session_id=session_id)
    bot.send_message(message.chat.id, ans)


# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    session_id = message.chat.id
    print(f"Answering on session {session_id}, msg={message.text}")
    answer = wine_agent(message.text, session_id=session_id)
    bot.send_message(message.chat.id, answer)

# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе


## Удаляем лишнее

В заключении удалим все созданные ресурсы. Для простоты мы удалим все ресурсы из текущего облака/проекта. 

**ВНИМАНИЕ**: Не выполняйте этот код, если у вас есть другие проекты с ассистентами в облаке!

In [ ]:
vector_stores = client.vector_stores.list()
for v in vector_stores:
    print(f" + Deleting vector store id={v.id}")
    client.vector_stores.delete(vector_store_id=v.id)

files = client.files.list(purpose='assistants')
for f in files:
    print(f" + Deleting file id={f.id}")
    client.files.delete(file_id=f.id)